In [12]:
import math
from itertools import groupby
from operator import itemgetter
import os

def split_coords(data, strand):
	# https://stackoverflow.com/questions/3149440/splitting-list-based-on-missing-numbers-in-a-sequence
	output = []
	if strand == "-" or strand == "-1":
		data.reverse()
	for k, g in groupby(enumerate(data), lambda i_x: i_x[0] - i_x[1]):
		output.append(list(map(itemgetter(1), g)))
	return output

directory = "../outputs/mutations/cds_bed_format/"
files = os.listdir(directory)

import sys
with open("../outputs/all_TFs_table_proteins.txt") as f1:
    f1.readline()
    for line in f1:
        #print(line)
        line = line.rstrip().split("\t")
        #print(line)
        tf, dbd_coords, ad_coords, rd_coords, bif_coords, length = line[3], line[6].split(","), line[7].split(","), line[8].split(","), line[9].split(","), len(line[-1])

        #print(tf)
        # if tf == "P37231":
        #     print("HI")
        ensg, enst = line[4], line[5]
        
        # Get coordinates of domains
        dbd_dom, ad_dom, rd_dom, bif_dom = [], [], [], []

        if dbd_coords != ["NA-NA"]:
            for coord in dbd_coords:
                coord = coord.split("-")
                start, end = int(coord[0]), int(coord[1])
                dbd_dom.append((start, end))
        if ad_coords != [""]:
            for coord in ad_coords:
                coord = coord.split("-")
                start, end = int(coord[0]), int(coord[1])
                ad_dom.append((start, end))
        if rd_coords != [""]:
            for coord in rd_coords:
                coord = coord.split("-")
                start, end = int(coord[0]), int(coord[1])
                rd_dom.append((start, end))
        if bif_coords != [""]:
            for coord in bif_coords:
                coord = coord.split("-")
                start, end = int(coord[0]), int(coord[1])
                bif_dom.append((start, end))

        all_doms = {"DBD": dbd_dom, "AD": ad_dom, "RD": rd_dom, "Bif": bif_dom}


        if enst in files and tf == "P37231":
        
            with open("../outputs/mutations/cds_bed_format/" + enst) as f2:
                coords = []
                nt_length = 0
                for line in f2:
                    line = line.rstrip().split("\t")
                    #print(line)
                    strand = line[4]
                    chrom = line[0]
                    # If the negative strand: switch the end and start of the exons  
                    if strand == "-" or strand == "-1":
                        #start, end = int(line[2]), int(line[1]) + 1
                        start, end = int(line[1]) + 1, int(line[2]) #SK 10/25/24
                    else:
                        start, end = int(line[1]), int(line[2]) + 1
    
                    coords.append((start, end))
                    nt_length += abs(end - start) + 1
    
                aa_length = int(nt_length/3)
                #if length != aa_length:
                    #print(tf, length, aa_length)
                    #continue
    
                n_exons = len(coords)
                #SK: commented next two lines out
    #             if strand == "-":
    #                 coords.reverse()
                # Sort in increasing order 
                if strand == "-":
                    coords.sort(key=lambda _: -1 * _[0])
                else:
                    coords.sort(key=lambda _: 1 * _[0])
    
                #print(tf, n_exons, strand)
                to_print = tf == "Q9Y261"

                dict_aa_coords = {}
                all_pos = []
                for start, end in coords:
                    if strand == "+" or strand == "1":
                        all_pos += list(range(start, end + 1))
                    elif strand == "-" or strand == "-1":
                        # all_pos += list(range(start, end -1, -1)) 
                        all_pos += list(range(end - 1, start, -1))  # SK change 10/25/24
                        # if to_print:
                        #     print('hi')
                        #     print(start, end)
                    else:
                        print("ERROR STRAND unrecognizable for " + tf)
                        
                start_codon = 0
                for i in range(aa_length):
                    end_codon = start_codon +3
                    dict_aa_coords[i+1] = all_pos[start_codon : end_codon]
                    start_codon += 3


                # if to_print:
                #     print(strand)
                #     print(type(strand))
                #     print(dict_aa_coords)
                #     print(all_pos)
                    
                with open("../outputs/mutations/domains_bed_format/" + tf, "w") as o1:
                    print(tf)


                    #if to_print:
                        #print(all_doms)
                    for name, domain in all_doms.items():
                        #if to_print:
                            #print(name)
                            #print(domain)
                        if domain == []:
                            continue
                        #print(name, domain)
                        for start_dom, end_dom in domain:
                            print(start_dom, end_dom)
                            #if to_print:
                                #print(start_dom, end_dom, domain)
                            dom_pos = []
                            for aa in range(start_dom, end_dom +1):
                                dom_pos += dict_aa_coords[aa]

                            #if to_print:
                                #print(dom_pos,strand)
                                
                            for cds_dom in split_coords(dom_pos, strand):
                                start_bed = cds_dom[0] - 1
                                end_bed = cds_dom[-1]
                                #print(str(chrom) + "\t" + str(start_bed) + "\t" + str(end_bed) + "\t" + name + "\t" + ensg + "\t.\t" + strand + "\t" + enst + "\n")

                                if strand == "-1":
                                    new_strand = "-"
                                else:
                                    new_strand = "+"
                                o1.write(str(chrom) + "\t" + str(start_bed) + "\t" + str(end_bed) + "\t" + name + "\t" + ensg + "\t.\t" + new_strand + "\t" + enst + "\n")
                                #print(tf, name, start_bed, end_bed)




P37231
137 205
12 101
495 503


In [23]:
import pandas as pd

tbl = pd.read_csv("../outputs/all_TFs_table_proteins.txt", sep = "\t")
tbl

Unnamed: 0   1   2 uniprotID  ENSG             ENST  \
0             0 NaN NaN    A6NJG6   NaN  ENST00000334384   
1             1 NaN NaN    A8MTJ6   NaN  ENST00000428390   
2             2 NaN NaN    A8MYZ6   NaN  ENST00000641094   
3             3 NaN NaN    A8MZ59   NaN  ENST00000640845   
4             4 NaN NaN    O00321   NaN  ENST00000403402   
..          ...  ..  ..       ...   ...              ...   
441         441 NaN NaN    Q9Y5R6   NaN  ENST00000382276   
442         442 NaN NaN    Q9Y5W3   NaN  ENST00000248071   
443         443 NaN NaN    Q9Y692   NaN  ENST00000294409   
444         444 NaN NaN    Q9Y6Q9   NaN  ENST00000371998   
445         445 NaN NaN    Q9Y6Y1   NaN  ENST00000303635   

                  DBD_coords        AD_coords  RD_coords  Bif_coords  length  
0                     79-135          142-315        NaN         NaN       1  
1                    145-234          369-420        NaN         NaN       1  
2                     89-178          382-491        NaN         NaN       1  
3                      31-65           92-181        NaN         NaN       1  
4                    242-321            1-162        NaN         NaN       1  
..                       ...              ...        ...         ...     ...  
441                   72-118          172-251        NaN         NaN       1  
442  272-296,302-326,332-354             1-89        NaN         NaN       1  
443                   89-165          422-573        NaN         NaN       1  
444                    31-83         621-1424        NaN         NaN       1  
445                   69-182  702-841,422-501        NaN         NaN       1  

[446 rows x 11 columns]

In [24]:
tbl[tbl["uniprotID"] == "Q9Y261"]

Unnamed: 0   1   2 uniprotID  ENSG             ENST DBD_coords  \
431         431 NaN NaN    Q9Y261   NaN  ENST00000377115    159-248   

        AD_coords  RD_coords  Bif_coords  length  
431  1-93,361-457        NaN         NaN       1